# Decorator
Decorators work as wrappers, modifying the behavior of the code before and after a target function execution, without the need to modify the function itself, augmenting the original functionality, thus decorating it.

## Basic knowledge about functions

In [3]:
# assign fucntions to variables
def greet(name):
    return('hello '+name)

greet_someone = greet
print(greet_someone('John'))

hello John


In [4]:
# Define functions inside other functions
def greet(name):
    def get_message():
        return 'hello '
    
    result=get_message()+name
    return result
print(greet('John'))

hello John


In [5]:
# Functions can be passed as parameters to otehr functions
def greet(name):
    return 'hello '+name

def call_func(func):
    other_name = 'John'
    return func(other_name)

print(call_func(greet))

hello John


In [6]:
# Functions can return other functions
def compose_greet_func():
    def get_message():
        return 'Hello there!'
    
    return get_message

greet = compose_greet_func()
print(greet())

Hello there!


In [8]:
# Inner funcitnos have access to the enclosing scope
def compose_greet_func(name):
    def get_message():
        return 'Hello there '+name+'!'
    
    return get_message

greet = compose_greet_func('John')
print(greet())

Hello there John!


## Composition of Decorators

In [10]:
def get_text(name):
    return "lorem ipsum, {0} dolor sit amet".format(name)

def p_decorate(func):
    def func_wrapper(name):
        return '<p>{0}</p>'.format(func(name))
    return func_wrapper

my_get_text = p_decorate(get_text)
print(my_get_text('John'))

<p>lorem ipsum, John dolor sit amet</p>


In [11]:
get_text = p_decorate(get_text)
print(get_text('John'))

<p>lorem ipsum, John dolor sit amet</p>


## Python's Decorator Syntax

In [15]:
def get_text(name):
    return "lorem ipsum, {0} dolor sit amet".format(name)

def p_decorate(func):
    def func_wrapper(name):
        return "<p>{0}</p>".format(func(name))
    return func_wrapper

def strong_decorate(func):
    def func_wrapper(name):
        return "<strong>{0}</strong>".format(func(name))
    return func_wrapper

def div_decorate(func):
    def func_wrapper(name):
        return "<div>{0}</div>".format(func(name))
    return func_wrapper
    
get_text = div_decorate(p_decorate(strong_decorate(get_text)))
print(get_text('John'))

<div><p><strong>lorem ipsum, John dolor sit amet</strong></p></div>


The above tedious functions can be achieved with much more expressive power using python decorator syntax

In [17]:
@div_decorate
@p_decorate
@strong_decorate
def get_text(name):
   return "lorem ipsum, {0} dolor sit amet".format(name)

print(get_text("John"))

<div><p><strong>lorem ipsum, John dolor sit amet</strong></p></div>


One important thing to notice here is that the order of setting our decorators matters. If the order was different in the example above, the output would have been different.


## Decorateing Methods
In Python, methods are functions that expect their first parameter to be a reference to the current object. We can build decorators for methods the same way, while taking self into consideration in the wrapper function.

In [18]:
def p_decorate(func):
   def func_wrapper(self):
       return "<p>{0}</p>".format(func(self))
   return func_wrapper

class Person(object):
    def __init__(self):
        self.name = "John"
        self.family = "Doe"

    @p_decorate
    def get_fullname(self):
        return self.name+" "+self.family

my_person = Person()
print(my_person.get_fullname())

<p>John Doe</p>


A much better approach would be to make our decorator useful for functions and methods alike. This can be done by putting args and *kwargs as parameters for the wrapper, then it can accept any arbitrary number of arguments and keyword arguments.

In [19]:
def p_decorate(func):
    def func_wrapper(*args, **kwargs):
        return "<p>{0}</p>".format(func(*args,**kwargs))
    return func_wrapper

class Person(object):
    def __init__(self):
        self.name = 'John'
        self.family = 'Doe'
        
    @p_decorate
    def get_fullname(self):
        return self.name+" "+self.family
    
my_person = Person()
print(my_person.get_fullname())

<p>John Doe</p>


## Passing auguments to decorators

In [22]:
def tags(tag_name):
    def tags_decorator(func):
        def func_wrapper(name):
            return "<{0}>{1}</{0}>".format(tag_name,func(name))
        return func_wrapper
    return tags_decorator

@tags("p")
def get_text(name):
    return "hello "+name

print(get_text('John'))

<p>hello John</p>


## Debugging decorated functions

In [23]:
print(get_text.__name__)

func_wrapper


In [25]:
from functools import wraps

def tags(tag_name):
    def tags_decorator(func):
        @wraps(func)
        def func_wrapper(name):
            return "<{0}>{1}</{0}>".format(tag_name,func(name))
        return func_wrapper
    return tags_decorator

@tags('p')
def get_text(name):
    """returns some text"""
    return 'hello '+name

print(get_text.__name__)
print(get_text.__doc__)
print(get_text.__module__)

get_text
returns some text
__main__


In [10]:
from functools import wraps

def add_wrapping_with_style(style):
    def add_wrapping(item):
        @wraps(item)
        def wrapped_item():
            return 'a {} wrapped up box of {}'.format(style,str(item()))
        return wrapped_item
    return add_wrapping

@add_wrapping_with_style('beautifully')
def new_gpu():
    return 'a new Tesla P100 GPU'

print(new_gpu())
print(new_gpu.__name__)

a beautifully wrapped up box of a new Tesla P100 GPU
new_gpu
